# OPCUA Bio Reactor Example


Start by importing the required packages:

In [1]:
import numpy as np
import do_mpc
import time
import opcua_wrapper

Next we define the model since we need it to initialize the server:

In [2]:
model_type = 'continuous' # either 'discrete' or 'continuous'
model = do_mpc.model.Model(model_type)
# States struct (optimization variables):
X_s = model.set_variable('_x',  'X_s')
S_s = model.set_variable('_x',  'S_s')
P_s = model.set_variable('_x',  'P_s')
V_s = model.set_variable('_x',  'V_s')
# Input struct (optimization variables):
inp = model.set_variable('_u',  'inp')
# Certain parameters
mu_m  = 0.02
K_m   = 0.05
K_i   = 5.0
v_par = 0.004
Y_p   = 1.2

# Uncertain parameters:
Y_x  = model.set_variable('_p',  'Y_x')
S_in = model.set_variable('_p', 'S_in')
# Auxiliary term
mu_S = mu_m*S_s/(K_m+S_s+(S_s**2/K_i))

# Differential equations
model.set_rhs('X_s', mu_S*X_s - inp/V_s*X_s)
model.set_rhs('S_s', -mu_S*X_s/Y_x - v_par*X_s/Y_p + inp/V_s*(S_in-S_s))
model.set_rhs('P_s', v_par*X_s - inp/V_s*P_s)
model.set_rhs('V_s', inp)
# Build the model
model.setup()

Now we can pass the model to the .Server() class so that it can create an OPCUA server for us. All information needed to create the so called "namespace" can be taken from the model, e.g. the number of states. Of course we also have to provide information like server address and port number.

In [3]:
# Defining the settings for the OPCUA server
server_opts = {"_model":model,                   # must be the model used to define the controller
               "_name":"Bio Reactor OPCUA",      # give the server whatever name you prefer
               "_address":"opc.tcp://localhost:4840/freeopcua/server/",  # does not need changing
               "_port": 4840,                    # does not need changing
               "_server_type": "with_estimator", # or select "basic" for no estimates
               "_store_params": True,            # should always be set to True to spare yourself the headaches
               "_store_predictions": False,      # not implemented yet
               "_with_db": True,                 # set to True if you plan to use the SQL database during/after the runtime 
               "_n_steps_pred": 20}

# Create your OPUA server
opc_server = opcua_wrapper.Server(server_opts)

Server setup #1: You have opted for the data server with state and parameter estimates!
Server setup #2: The model parameters will be stored on the server.
Server setup #3: The OPCUA server will not have the predictions available at runtime.


Great, now we can start the server:

In [4]:
opc_server.start()

Endpoints other than open requested but private key and certificate are not set.
Listening on localhost:4840


SQL database error, the following node can not be historized:
 ns=2;s=PredictedStates
SQL database error, the following node can not be historized:
 ns=2;s=Estimates.Z


True

Before we proceed, let's take a quick look at the namespace created for our server.

In [5]:
opc_server.namespace

{'PlantData': {'x': 'States.X',
  'z': 'States.Z',
  'u': 'Inputs',
  'y': 'Measurements',
  'p': 'Parameters'},
 'ControllerData': {'x_init': 'InitialState',
  'u_opt': 'OptimalOutputs',
  'x_pred': 'PredictedStates',
  'u_pred': 'PredictedOutputs'},
 'EstimatorData': {'xhat': 'Estimates.X',
  'zhat': 'Estimates.Z',
  'phat': 'Estimates.P'},
 'SupervisionData': {'flags': 'Flags', 'switches': 'Switches'}}

Ok, now we need to tell the client where to find the server. Note that we can just take the namespace from the opc_server, no need to type it by hand...

In [6]:
client_opts = {"_address":"opc.tcp://localhost:4840/freeopcua/server/", # the basic implementation, can remain as is
               "_port": 4840,                              # should remain as is
               "_client_type": "ManualUser",               # simulator, estimator, user
               "_namespace": opc_server.namespace}         # must match the server, therefore simply copy the namespace 

Next we initialize the controller. The trick is that we don't use the normal do-mpc controller, but the opc_controller. But since this inherits all methods from the do-mpc controller, the setup is identical. 
But before we setup the mpc, we give it the client information that we already defined:

In [7]:
control_opts = {}
control_opts['_opc_opts'] = client_opts
control_opts['_cycle_time'] = 10.0      # the cycle time for the asynchronous step
control_opts['_output_feedback'] = True # defined wether the controller uses estimator or simulator data
control_opts['_user_controlled'] = False
control_opts['_opc_opts']['_client_type'] = 'controller'

mpc = opcua_wrapper.RealtimeController(model,control_opts)

A client of the type - controller - was created
The - controller - has just connected to  opc.tcp://localhost:4840/freeopcua/server/


Let's setup the mpc. This should be familiar.

In [8]:
setup_mpc = {
    'n_horizon': 20,
    'n_robust': 1,
    'open_loop': 0,
    't_step': 1.0,
    'state_discretization': 'collocation',
    'collocation_type': 'radau',
    'collocation_deg': 2,
    'collocation_ni': 2,
    'store_full_solution': True,
    # Use MA27 linear solver in ipopt for faster calculations:
    #'nlpsol_opts': {'ipopt.linear_solver': 'MA27'}
}

mpc.set_param(**setup_mpc)

mterm = -model.x['P_s'] # terminal cost
lterm = -model.x['P_s'] # stage cost

mpc.set_objective(mterm=mterm, lterm=lterm)
mpc.set_rterm(inp=1.0) # penalty on input changes

# lower bounds of the states
mpc.bounds['lower', '_x', 'X_s'] = 0.0
mpc.bounds['lower', '_x', 'S_s'] = -0.01
mpc.bounds['lower', '_x', 'P_s'] = 0.0
mpc.bounds['lower', '_x', 'V_s'] = 0.0

# upper bounds of the states
mpc.bounds['upper', '_x','X_s'] = 3.7
mpc.bounds['upper', '_x','P_s'] = 3.0

# upper and lower bounds of the control input
mpc.bounds['lower','_u','inp'] = 0.0
mpc.bounds['upper','_u','inp'] = 0.2

Y_x_values = np.array([0.5, 0.4, 0.3])
S_in_values = np.array([200.0, 220.0, 180.0])

mpc.set_uncertainty_values(Y_x = Y_x_values, S_in = S_in_values)

mpc.setup()

This procedure is reapeated for the simulator:

In [9]:
sim_opts = {}
sim_opts['_opc_opts'] = client_opts
sim_opts['_cycle_time'] = 2
sim_opts['_user_controlled'] = False
sim_opts['_opc_opts']['_client_type'] = 'simulator'

simulator = opcua_wrapper.RealtimeSimulator(model,sim_opts)

A client of the type - simulator - was created
The - simulator - has just connected to  opc.tcp://localhost:4840/freeopcua/server/


In [10]:
params_simulator = {
    'integration_tool': 'cvodes',
    'abstol': 1e-10,
    'reltol': 1e-10,
    't_step': 1.0
}

simulator.set_param(**params_simulator)

p_num = simulator.get_p_template()

p_num['Y_x'] = 0.4
p_num['S_in'] = 200.0

# function definition
def p_fun(t_now):
    return p_num

# Set the user-defined function above as the function for the realization of the uncertain parameters
simulator.set_p_fun(p_fun)

simulator.setup()


And for the estimator (which is not implemented yet... so state feedback it is):

In [11]:
est_opts = {}
est_opts['_opc_opts'] = client_opts
est_opts['_cycle_time'] = 4
est_opts['_user_controlled'] = False
est_opts['_opc_opts']['_client_type'] = 'estimator'
est_opts['_opc_opts']['_output_feedback'] = True

estimator = opcua_wrapper.RealtimeEstimator('state-feedback',model,est_opts)

A client of the type - estimator - was created
The - estimator - has just connected to  opc.tcp://localhost:4840/freeopcua/server/


Next we need to write some values to the server. But first we need a initial state:

In [12]:
# Initial state
X_s_0 = 1.0 # Concentration biomass [mol/l]
S_s_0 = 0.5 # Concentration substrate [mol/l]
P_s_0 = 0.0 # Concentration product [mol/l]
V_s_0 = 120.0 # Volume inside tank [m^3]
x0 = np.array([X_s_0, S_s_0, P_s_0, V_s_0])

In [13]:
# Step 1: initilize the simulator part
simulator.x0 = x0
simulator.init_server() #soll keinen input brauchen weil das ja klar ist...

# Step 2: initialize the estimator part (if present)
estimator.x0 = x0
estimator.init_server()

# Step 3: only now can the optimizer be initialized, and a first optimization can be executed
time.sleep((simulator.cycle_time+estimator.cycle_time)/2)
mpc.x0 = x0
mpc.init_server()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    20164
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     7543

Total number of variables............................:     5472
                     variables with only lower bounds:     2412
                variables with lower and upper bounds:     2592
                     variables with only upper bounds:        0
Total number of equa

True

Now the setup is complete and we can begin asynchronous operation:

In [14]:
"""
Define triggers for each of the modules and start the parallel/asynchronous operation
"""
trigger_simulator  = opcua_wrapper.RealtimeTrigger(simulator.cycle_time , simulator.asynchronous_step)

trigger_estimator  = opcua_wrapper.RealtimeTrigger(estimator.cycle_time , estimator.asynchronous_step)

trigger_controller = opcua_wrapper.RealtimeTrigger(mpc.cycle_time, mpc.asynchronous_step)
#leichte möglichkeit den server auszulesen


time.sleep(20)

trigger_controller.stop()
trigger_simulator.stop()
trigger_estimator.stop()

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    20164
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     7543

Total number of variables............................:     5472
                     variables with only lower bounds:     2412
                variables with lower and upper bounds:     2592
                     variables with only upper bounds:        0
Total number of equality constraints.................:     5044
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

We can now use the .get_data() method to accsess the SQL database.

In [15]:
df_x, df_u = opc_server.get_data()
df_x

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    20164
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     7543

Total number of variables............................:     5472
                     variables with only lower bounds:     2412
                variables with lower and upper bounds:     2592
                     variables with only upper bounds:        0
Total number of equality constraints.................:     5044
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

,s=Measurements_0,s=Measurements_1,s=Measurements_2,s=Measurements_3,Timestamp
0,1.157766,0.210779,0.038786,120.083825,2023-01-26 09:16:54.292982
1,1.139778,0.237524,0.034195,120.070375,2023-01-26 09:16:52.301021
2,1.121807,0.264160,0.029676,120.056925,2023-01-26 09:16:50.293589
3,1.103923,0.290517,0.025227,120.043476,2023-01-26 09:16:48.314663
4,1.086175,0.316467,0.020850,120.030026,2023-01-26 09:16:46.384427
...,...,...,...,...,...
235,1.086175,0.316467,0.020850,120.030026,2023-01-22 11:57:03.183611
236,1.068515,0.354355,0.016541,120.024021,2023-01-22 11:57:01.155809
237,1.051033,0.391736,0.012303,120.018015,2023-01-22 11:56:59.152071
238,1.033770,0.428512,0.008134,120.012010,2023-01-22 11:56:57.180998


It is important to disconnect the clients again and stop the server!

In [16]:
mpc.stop()
simulator.stop()
estimator.stop()
opc_server.stop()

A client of type controller disconnected from server opc.tcp://localhost:4840/freeopcua/server/
A client of type simulator disconnected from server opc.tcp://localhost:4840/freeopcua/server/
A client of type estimator disconnected from server opc.tcp://localhost:4840/freeopcua/server/
The server  Bio Reactor OPCUA was stopped successfully @  2023-01-26 10:17 Mitteleuropäische Zeit


True